<a href="https://colab.research.google.com/github/martysteer/omftm/blob/master/OMFTM_Gwern_GPT_2_Gutenberg_Poetry_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Our Mutual Friend the Machine
### [Being Human Festival](https://beinghumanfestival.org/event/our-mutual-friend-the-machine/), November 2019

Run the cells below to setup the environment. You need to connect the notebook to your google drive, then run the cells in order. This will download the metadata and model files from github into your Google drive.

This notebook uses [Gwern's Gutenberg Poetry model](https://www.gwern.net/GPT-2), which was trained on the [Gutenburg Poetry Corpus](https://doi.org/10.3389/fdigh.2018.00005) using [OpenAI's GPT-2 Transformer](https://openai.com/blog/better-language-models/) architecture for Natural Language Generation (NLG). This notebook was based on [Max Wolf's Google Colab notebook](https://minimaxir.com/2019/09/howto-gpt2/).

The gutenburg metadata has been filtered to 19th Century authors/volumes.

Enjoy!

[Digital Humanities @SAS](https://www.sas.ac.uk/projects-and-initiatives/digital-humanities)


##  Libraries and setup


In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

import re
import json
import random

In [0]:
!nvidia-smi

In [0]:
gpt2.mount_gdrive()

In [0]:
# Copy the metadata into your google drive. This will overwrite any existing file
!wget https://github.com/martysteer/omftm/releases/download/Gwern-GPT-2-345M/gutenbermetadata.json -O ./drive/My\ Drive/gutenbermetadata.json

In [0]:
# Copy the Gwern GPT-2 poetry model into your google drive. This will overwrite any existing file
!wget https://github.com/martysteer/omftm/releases/download/Gwern-GPT-2-345M/checkpoint_345M_poetry.tar -O ./drive/My\ Drive/checkpoint_345M_poetry.tar

In [0]:
# This extracts 19th century author metadata and filters it a bit.

# Copy from your google drive into the colab environment
!cp  ./drive/My\ Drive/gutenbermetadata.json .

with open('gutenbermetadata.json') as f:
  metadata = json.load(f)


# Save a reference list of 19th century authors
ninteethnc = {}
for item in metadata:
    if 'Author Death' in item:
        death = item['Author Death']
        if len(death) > 0 and isinstance(item['Author Death'][0], int) and item['Author Death'][0] < 1900:
            ninteethnc[int(item['Num'])] = item
            
with open('ninteenth-century-gutenberg-authors.txt', 'w') as f:
    for gid, item in ninteethnc.items():
        print(item['Num'], item['Author'][0], file=f)


# Save a reference list of 19th century authors and their works
auths = {}
for aid, item in ninteethnc.items():
    auths.setdefault(item['Author'][0],[]).append(item)
    
with open('ninteenth-century-gutenberg.txt', 'w') as f:
    for aid, items in auths.items():
        print(aid, file=f)
        for item in items:
            print(item['Num'], item['Title'][0], file=f)
        print('', file=f)
        print('', file=f)


def gidsummary(gid):
  auth = ninteethnc[gid]
  out = ['=========', auth['Title'][0], auth['Author'][0]]
  if 'LoC Class' in auth.keys() and len(auth['LoC Class']):
    out += [auth['LoC Class'][0]]
  return out

Setup the logging. All poems which are generated and the input parameters are logged to google drive.

In [0]:
import logging
import uuid
import os

# Each time you run the notebook we generate a UUID and use this is the logfiles.
notebookuuid = str(uuid.uuid4())

logdir = "./drive/My Drive/logs/"
if not os.path.exists(logdir):
    os.makedirs(logdir)

logging.basicConfig(filename=logdir + notebookuuid + '.txt',
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

gpt2poetrylogger = logging.getLogger('gpt-2-poetry' + notebookuuid)

In [0]:
def formatGPToutput(samplelist, numPoems=10, logger=None):
  '''
  This formats the output. Strips the gutenburg id from the start of each line
  and looks up this author's metadata.
  '''
  if not logger:
    logger = logging.getLogger('gpt-2-poetry' + notebookuuid)
  
  for poem in random.sample(samplelist, numPoems):
      alllines = re.findall('([0-9]+)?\|?(.*)', poem)
      gid = int(list(set(g[0] for g in alllines if g[0]))[0])
      lines = [l[1] for l in alllines if l[1]]

      logger.info('Gutenburg ID: ' + str(gid))
      
      # If it is 19th C, print poem, else skip.
      # The metadata will
      if gid in ninteethnc.keys():
        logger.info(gidsummary(gid))
        print('\n\r'.join(gidsummary(gid)))
        print('---')
        print('\n\r'.join(lines), '...')
        print()
      else:
        logger.info('Not C19th')
        
      # Log all poems
      logger.info(lines)

In [0]:
def generatePoetry(seedLine, authorid=None, numPoems=10, poemlength=100, logger=None):
  '''
  This does what it says, but it also logs the parameters which 
  were used to generate the poetry.
  '''
  if not logger:
    logger = logging.getLogger('gpt-2-poetry' + notebookuuid)
  
  logger.info('========================')
  logger.info('Running generatepoetry()')
  logger.info('Seedline: ' + seedLine)
  logger.info('Author GID: ' + authorid)
  logger.info('Poem length: '+ str(poemlength))
  logger.info('Num Poems: '+ str(numPoems))
  
  if authorid:
    seedLine = str(authorid) + '|' + seedLine
    
  samples = gpt2.generate(sess,
              length=poemlength,
              temperature=0.95,
              run_name=run_name,
              prefix=seedLine,
              return_as_list=True,
              nsamples=10,  # always 10, but the format function will limit
              batch_size=5,
              top_k=40, 
  #               top_p=0.4
              )
  return formatGPToutput(samples, numPoems, logger)

## Load a Trained Model Checkpoint

Running the next cell will copy the `.tar` checkpoint file from Google Drive into the Colaboratory VM. The 

In [0]:
poetryprefix = "345M_poetry"
run_name = poetryprefix
gpt2.copy_checkpoint_from_gdrive(run_name=run_name)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name)

## GPT-2 19th Century Literature poem generator
Choose from the list of 19th century project gutenburg poets and generate some poetry! Try to use the last line of one poem as the starting line of the second poem, and experiment with different styles.

Copy/Paste the ones you like, edit them, clean them up a little bit, change some words here and there, and make it your own.

In [0]:
#@markdown ---
#@markdown ### Configure your poem:
Poem_start = "Seaside as evening, raining" #@param {type:"string"}
Poem_length = "100" #@param ['20','50','100','150','200','300','500','1000']
Number_of_poems = "10" #@param ['1', '2', '5', '10', '20']
In_the_style_of = "4212 Matthew Arnold, Culture and Anarchy" #@param ["4212 Matthew Arnold, Culture and Anarchy", "5159 Matthew Arnold, Celtic Literature", "27739 Matthew Arnold, Poetical Works of Matthew Arnold", "4253 Robert Browning, Dramatic Romances", "16376 Robert Browning, Browning's Shorter Poems", "18343 Robert Browning, The Pied Piper of Hamelin", "50954 Robert Browning, The Complete Poetic and Dramatic Works of, Cambridge Edition", "11 Lewis Carroll, Alice's Adventures in Wonderland", "651 Lewis Carroll, Phantasmagoria and Other Poems", "4763 Lewis Carroll, The Game of Logic", "28696 Lewis Carroll, Symbolic Logic", "33582 Lewis Carroll, Rhyme? And Reason?", "8209 John Keats, Poems 1817", "23684 John Keats, Keats: Poems Published in 1820", "24280 John Keats, Endymion A Poetic Romance", "51120 John Keats, Poèmes et Poésies Traduction précédée d'une étude par Paul Gallimard", "982 Edward Lear, The Book of Nonsense", "13649 Edward Lear, Laughable Lyrics", "20113 Edward Lear, Nonsense Drolleries The Owl & The Pussy-Cat—The Duck & The Kangaroo.", "34906 Edward Lear, The Jumblies, and Other Nonsense Verses", "4654 Percy Bysshe Shelley, The Daemon of the World", "4696 Percy Bysshe Shelley, The Witch of Atlas", "4800 Percy Bysshe Shelley, The Complete Poetical Works of", "5428 Percy Bysshe Shelley, A Defence of Poetry and Other Essays", "1322 Walt Whitman, Leaves of Grass", "8388 Walt Whitman, Poems By", "8801 Walt Whitman, Drum Taps", "8813 Walt Whitman, Complete Prose Works Specimen Days and Collect, November Boughs and Goodbye My Fancy", "27494 Walt Whitman, The Patriotic Poems of", "47846 Walt Whitman, Poèmes de"]





generatePoetry(Poem_start, 
               authorid=In_the_style_of.split(' ')[0], 
               poemlength=int(Poem_length), 
               numPoems=int(Number_of_poems))

(This license came from [the original notebook, which was released by Max Wolf](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce)).

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.